In [ ]:
import numpy as np
import sys
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm
from anytree import RenderTree
from itertools import filterfalse
from functools import reduce

#### the same as above but with eager execution enabled

In [ ]:
import numpy as np
import sys
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm

In [ ]:
CODE_MSB_VALUE = 1<<3
def codeBoard(board):
    return np.array(list(map(
        lambda L:list(map(lambda l: [int(x) for x in bin((1<<l)+CODE_MSB_VALUE)[3:]], L)),
        board
    )))

In [ ]:
def getRot3(board):
    rotations = [board]
    for i in range(3):
        rotations.append(np.rot90(rotations[-1]))
    return rotations

def augment(board):
    flip = np.transpose(board)
    return [codeBoard(v) for v in getRot3(board)+getRot3(flip)]

In [ ]:
board = [[0,0,1,2],
         [1,1,2,0],
         [0,0,2,1],
         [1,2,0,0]
        ]

code = np.array(augment(board), dtype=np.float32)
segment_hight = code.shape[1]
segment_width = code.shape[2]
code_depth = code.shape[3]

In [ ]:
l = getVacationsIterator(board)
list(l)

#### actions iterator

In [ ]:
def getVacationsIterator(board):
    return zip(*np.nonzero(np.array(board) == 0))

orig_dirs = [(-1,0),(-1,-1),(0,-1),(1,-1)]
SingleDim = 4
MIN_LINE_SIZE = 4
MIN_LINE_SIZE_M1 = MIN_LINE_SIZE - 1
isInRange = lambda loc: np.all(np.array(loc)>=0) and np.all(np.array(loc)<SingleDim)

flip = lambda p: [0,2,1][p]

def isStraitConnection(board, location, player):
    connection = []
    for shift in orig_dirs:
        line = []
        
        loc = location
        while True:
            loc = (loc[0]+shift[0], loc[1]+shift[1])
            if not isInRange(loc) or board[loc] != player: break
            line.append(loc)
        #print(line)
            
        loc = location
        while True:
            loc = (loc[0]-shift[0], loc[1]-shift[1])
            if not isInRange(loc) or board[loc] != player: break
            line.append(loc)
        #print(line)
        
        if len(line) >= MIN_LINE_SIZE_M1:
            return True
    
    return False

def getOneMoveTransition(board, move):
    player, location = move
    is_terminal = isStraitConnection(board, location, player)
    next_board = board.copy()
    next_board[location] = player
    return (is_terminal, next_board)

def applyMove(board, move):
    location, player = move
    next_board = board.copy()
    next_board[location] = player
    return (is_terminal, next_board)


<font color=#404040>
Early draw detection
</font>

In [ ]:
def getLocationsOf(board, mark):
    return zip(*np.nonzero(np.array(board) == mark))

board_shape = (SingleDim,SingleDim)
dir_inc = np.array([[0,1],[1,1],[1,0],[1,-1]])
# bit set of winning configurations a board cell is part of, built for each board cell
bit_sets = np.zeros(shape=board_shape, dtype=int)
bit = 0 # codes a distinct winning configuration
for row,col in np.ndindex(board_shape):
    for row_inc,col_inc in dir_inc:
        end = (row+row_inc*MIN_LINE_SIZE_M1, col+col_inc*MIN_LINE_SIZE_M1)
        if not end in np.ndindex(board_shape):
            continue
        for i in range(MIN_LINE_SIZE):
            bit_sets[row+row_inc*i, col+col_inc*i] |= 1<<bit
        bit += 1
complete_bit_set = \
    reduce(lambda a,b: a|b, [bit_sets[x] for x in np.ndindex(board_shape)])
bit, complete_bit_set

In [ ]:
def isDraw(board):
    return \
        reduce(lambda a,b: a|b, [bit_sets[x] for x in getLocationsOf(board,2)]) == complete_bit_set \
        and reduce(lambda a,b: a|b, [bit_sets[x] for x in getLocationsOf(board,1)]) == complete_bit_set

In [ ]:
bit

In [ ]:
board = [[0,2,1,0],
         [1,1,2,0],
         [2,1,0,1],
         [0,0,1,2]
        ]

isDraw(board)

#### test the environment

In [ ]:
[(getOneMoveTransition(np.array(board),(2,loc))[0], loc) for loc in getVacationsIterator(board)]

#### test Conv2D

In [ ]:
k2D = np.array([[[2, 1, 0],[6, 0, 1],[2, 1, 0],[0,-1, 1]],
               [[2, 1, 3],[9,-7, 1],[2, 1,-9],[5, 0, 0]],
               [[0, 1, 3],[0, 0, 1],[2, 1,-3],[5,-1, 1]],
               [[2, 1,-3],[0,-2, 1],[2, 1, 3],[4, 0,11]]
              ], dtype=np.float32)
kernel2D = tf.reshape(k2D, k2D.shape+(1,), name='kernel2D')

#reshape for 2D convolution
code2D = code.reshape((1,code.shape[0]*code.shape[1],)+code.shape[2:])
print(tf.squeeze(tf.nn.conv2d(code2D, filters=kernel, strides=4, padding='VALID')).numpy())

#### test Conv1D

In [ ]:
k1D = np.array([[2, 1, 3],[9, 0, 1],
                [0,-1, 4],[5, 0,-3]], dtype=np.float32)
kernel1D = tf.reshape(k1D, k1D.shape+(1,), name='kernel1D')
#reshape for 1D convolution
code1D = code.reshape((1,code.shape[0]*code.shape[1]*code.shape[2],code.shape[3]))
print(tf.squeeze(tf.nn.conv1d(code1D, filters=kernel1D, stride=4, padding='VALID')).numpy())

#### build the model

In [ ]:
class vModel(tf.keras.Model):
    def __init__(self):
        super(vModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=44, name='dense1')
        self.dense2 = tf.keras.layers.Dense(units=1, name='dense2')
    
    def call(self, input):
        x = self.dense1(input)
        x = self.dense2(x)
        return tf.math.reduce_max(x, axis=0)

model = vModel()

#### get the gradient

In [ ]:
def grad(position, estimate):
    with tf.GradientTape() as t:
        y = model(position)
        loss = tf.square(y, estimate)
        grad = t.gradient(loss, model.trainable_variables)

### Proof number search
#### the class ProofNumberNode

In [ ]:
INF = sys.maxsize
class ProofNumberNode:
    def __init__(self, is_and, pn=1, dn=1, is_term=False):
        self.is_and = is_and
        self.expanded = is_term
        self.proof_num = pn
        self.disproof_num = dn
        self.children = []

    def getNumber(self):
        if self.is_and:
            return self.proof_num
        else:
            return self.disproof_num
        
    def isAnd(self):
        return self.is_and
    
    def isExpanded(self):
        return self.expanded
    
    def update(self):
        if len(self.children) == 0: return
        if not self.is_and:
            self.disproof_num = 0
            self.proof_num = INF
            for child in self.children:
                self.disproof_num += child.disproof_num
                self.proof_num = min(child.proof_num, self.proof_num)
        else:
            self.proof_num = 0
            self.disproof_num = INF
            for child in self.children:
                self.proof_num += child.proof_num
                self.disproof_num = min(child.disproof_num, self.disproof_num)

    def expand(self):
        if self.expanded: return
        self.expanded = True
        it = self.getExpandIterator()
        for child in it:
            shortcut = child.proof_num == 0 and not self.is_and or \
               child.disproof_num == 0 and self.is_and
            self.children.append(child)
            if shortcut: break
                
    def __repr__(self):
        return "<Type: "+["OR","AND"][self.is_and]+"; pn:"+str(self.proof_num)+ \
            "; dn:"+str(self.disproof_num)+"; "+["","expd "][self.expanded]+ \
            str(len(self.children))+" ch>"

#### Search implementation

In [ ]:
def descendToMPN(node):
    if not node.isExpanded():
        node.expand()
        node.update()
        return True
    
    select = []
    if node.isAnd():
        dn = INF
        for child in node.children:
            if child.disproof_num < dn:
                dn = child.disproof_num
                select = [child]
            # no need anymore:
            #elif child.disproof_num == dn: 
            #    select.append(child)
    else:
        pn = INF
        for child in node.children:
            if child.proof_num < pn:
                pn = child.proof_num
                select = [child]
            # no need anymore:
            #elif child.proof_num == pn:
            #    select.append(child)
    expanded = False
    for child in select:
        expanded = descendToMPN(child)
        if expanded: break

    node.update()
    return expanded

def iteratePNSearch(root, max_iterations=None, max_nodes=None):
    n=0
    while (not max_nodes or root.count < max_nodes) and \
        (not max_iterations or n < max_iterations):
        if not descendToMPN(root):
            print("\nNo expansion. Search terminated.")
            break
        if root.proof_num == 0:
            print("\nprooved")
            break
        elif root.disproof_num == 0:
            print("\ndisprooved")
            break
        else:
            n += 1
            print("\rIteration {:4}: nodes count is {:5}".format(n,root.count), 
                  flush=True, end='')
    print("\nIteration {:4}: nodes count is {:5}".format(n,root.count))

### Test PNS

In [ ]:
class TestPNS(ProofNumberNode):
    def __init__(self, is_and, min_children=0):
        self.ch_num = 0
        if np.random.rand() < 0.15:
            pn = 0 
            dn = INF
        elif np.random.rand() < 0.25:
            pn = INF 
            dn = 0
        else:
            self.ch_num = np.random.randint(min_children,5)
            if self.ch_num == 0:
                pn = INF//2
                dn = INF//2
            else:
                pn = 1
                dn = 1
        ProofNumberNode.__init__(self, is_and, pn, dn, self.ch_num==0)
        TestPNS.count += 1

    def getExpandIterator(self):
        for i in range(self.ch_num):
            yield TestPNS(not self.is_and)

class rootPNS(TestPNS):
    def __init__(self, is_and):
        ProofNumberNode.__init__(self, is_and, 1, 1)
        self.ch_num = np.random.randint(2,5)
        TestPNS.count = 1


#### Run test search with maximum 100 expanded nodes

In [ ]:
root = rootPNS(False)
iteratePNSearch(root)
print(root.count)

In [ ]:
root.children[1].children[7].is_and

<font color=#307030>
Render the tree, skipping unexpanded nodes:
</font>

In [ ]:
print(RenderTree(root, childiter=lambda chld: filterfalse(lambda i: not i.isExpanded(), chld)))
#print(RenderTree(root))

In [ ]:
print(descendToMPN(root.children[0]))
#root.children[2].update()
root.update()

#### Continue test search, if previosely stopped by exceeding maximum number of nodes

In [ ]:
iteratePNSearch(root, max_nodes=1000)

### TicTacToe PNS implementation

In [ ]:
class TicTacToeNode(ProofNumberNode):
    def __init__(self, is_and, position, depth):
        is_term = depth == SingleDim**2
        self.depth = depth
        board, loc, player = position
        self.board = board.copy()
        self.loc = loc
        self.board[loc] = player
        self.player = player
        if isStraitConnection(board, loc, player):
            if is_and:
                pn = 0
                dn = INF
            else:
                pn = INF
                dn = 0
            is_term = True
        else:
            is_term = isDraw(self.board)
            if is_term:
                pn = INF//2
                dn = INF//2
            else:
                pn = 1
                dn = 1

        ProofNumberNode.__init__(self, is_and, pn, dn, is_term)
        TicTacToeNode.count += 1

    def getExpandIterator(self):
        for loc in getVacationsIterator(self.board):
            yield TicTacToeNode(not self.is_and, 
                                (self.board, loc, flip(self.player)), 
                                self.depth+1)
    def __repr__(self):
        INF2 = INF//2
        pn = ["", "pn="+str(self.proof_num)+"; "][self.proof_num < INF2]
        dn = ["", "dn="+str(self.disproof_num)+"; "][self.disproof_num < INF2]
        stat = [" "," WIN"," LOOSE"," DRAW"][
                (self.proof_num==0)+((self.disproof_num==0)<<1)+
                (self.proof_num>=INF2 and self.disproof_num>=INF2)*3
        ]
        return "<"+str(self.player)+":"+stat+"; "+pn+dn+">"

In [ ]:
'''   
    def __repr__(self):
        return "<"+str(self.player)+": "+["","expd "][self.expanded]+ \
            str(len(self.children))+" ch; "+str(self.loc)+ \
            [" "," WIN"," LOOSE"][(self.proof_num==0)+((self.disproof_num==0)<<1)]+">"
    def __repr__(self):
        return "<"+str(self.player)+" "+str(self.board)+["","expd "][self.expanded]+ \
            str(len(self.children))+" ch>"
'''

### Implementation of transposition table

In [ ]:
class PlaySetHash():
    def __init__(self, key):
        self.key = key
        self.hash = reduce(lambda a,b: a^b, map(hash, key), 0)
    def __hash__(self):
        return self.hash
    def __eq__(self, another):
        return set(self.key) == set(another.key)

class TranspositionNode(TicTacToeNode):
    def __init__(self, is_and, position, depth, playset):
        self.playset = playset
        TicTacToeNode.__init__(self, is_and, position, depth)

    def getExpandIterator(self):
        for loc in getVacationsIterator(self.board):
            player = flip(self.player)
            playset = PlaySetHash(self.playset.key+[(loc,player)])
            node = TranspositionNode.table.get(playset)
            if not node:
                node = TranspositionNode(not self.is_and,
                                (self.board, loc, player), 
                                self.depth+1, 
                                playset)
                TranspositionNode.table[playset] = node
            yield node                

    def __repr__(self):
        INF2 = INF//2
        pn = ["", "pn="+str(self.proof_num)+"; "][self.proof_num < INF2]
        dn = ["", "dn="+str(self.disproof_num)+"; "][self.disproof_num < INF2]
        return "<"+str(self.player)+": "+str(list(zip(*self.playset.key))[0])+ \
            [" "," WIN"," LOOSE"," DRAW"][
                (self.proof_num==0)+((self.disproof_num==0)<<1)+
                (self.proof_num>=INF2 and self.disproof_num>=INF2)*3
            ]+pn+dn+">"

### Run TicTacToe PNS

In [ ]:
test_board = np.array(
        [[0,0,2,0],
         [0,1,1,0],
         [0,1,2,0],
         [0,0,0,0]
        ], dtype = int)
TicTacToeNode.count = 0
root = TicTacToeNode(False, (test_board, (3,3), 2), 6)
iteratePNSearch(root, max_iterations=10000, max_nodes=70000)

In [ ]:
test_board = np.array(
        [[0,0,0,0],
         [0,1,0,0],
         [0,1,2,0],
         [0,0,0,0]
        ], dtype = int)
TicTacToeNode.count = 0
root = TicTacToeNode(False, (test_board, (0,2), 2), 4)
iteratePNSearch(root, max_nodes=200000)

In [ ]:
test_board = np.array(
        [[0,0,2,0],
         [0,1,1,0],
         [0,1,2,0],
         [0,0,0,0]
        ], dtype = int)
TicTacToeNode.count = 0
TranspositionNode.table = {}
trp_root = TranspositionNode(False, (test_board, (3,3), 2), 6, PlaySetHash([((0,2),2)]))
iteratePNSearch(trp_root, max_nodes=50000)

In [ ]:
print(RenderTree(root, childiter=lambda chld: filterfalse(lambda i: not i.isExpanded(), chld)))

In [ ]:
root.children

<font color=#404040>
Some testing stuff
</font>

In [ ]:
ps1 = PlaySetHash([354,789,302])
ps2 = PlaySetHash([789,302,354])
ps3 = PlaySetHash([789,302,354,1])
d = {}
d[ps1] = ps1.key
d[ps2] = ps2.key
d[ps3] = ps3.key

In [ ]:
True*3